In [1]:
import pandas as pd
import sys
sys.path.append('../')
from evaluation.generated_dataset import GeneratedDataset

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/matminer/utils/data.py:326: UserWarning: MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.
  warnings.warn(f"{self.__class__.__name__}(impute_nan=False):\n" + IMPUTE_NAN_WARNING)


In [2]:
mp_20_source_sample = GeneratedDataset.from_transformations(("split", "test", "1k-sample"), sort=False).data

In [3]:
diffcsp_sample = GeneratedDataset.from_cache(("split", "test", "1k-sample", "WyckoffReduction", "DiffCSP++")).data
diffcsp_sample.index = mp_20_source_sample.index

In [4]:
matching_sg = (mp_20_source_sample.spacegroup_number == diffcsp_sample.spacegroup_number)
print(f"Matching spacegroup numbers: {matching_sg.sum()} out of {len(matching_sg)}")

Matching spacegroup numbers: 989 out of 1000


In [5]:
from pymatgen.analysis.structure_matcher import StructureMatcher
sm = StructureMatcher()
structure_matches = pd.Series(
    [sm.fit(s1, s2) for s1, s2 in zip(diffcsp_sample.structure, mp_20_source_sample.structure)],
    index=diffcsp_sample.index
)

print(f"Matching structures: {structure_matches.sum()} out of {len(structure_matches)}")

Matching structures: 776 out of 1000


In [6]:
import json
json_sample = diffcsp_sample.loc[~structure_matches].structure.map(lambda s: json.dumps(s.as_dict()))
json_sample.name = "json"
json_sample.to_csv("../generated/mp_20/split/test/1k-sample/WyckoffReduction/DiffCSP++/diffcsp_mismatches.csv.gz",
    index_label="mp_20_sequence_id")